# **Выгрузка данных с Opentable**

Настройка Selenium

In [15]:
!pip install selenium

In [16]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,555 B in 2s (3,568 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [17]:
!apt-get install chromium-driver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [20]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-blink-features=AutomationControlled")  # Скрываем автоматизацию
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=options)
    return driver

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [24]:
!pip install bs4

In [25]:
import requests
from bs4 import BeautifulSoup

## **Выгрузка данных со страницы со списком ресторанов**

Настройка логирования

In [40]:
import logging

logging.basicConfig(
    filename='opentable_rest.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)
logging.info('Тест')

Пагинация через Selenium, остальное через BeautifulSoup

In [41]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
}

base_url = "https://www.opentable.co.uk/open/london"

restaurants_data = []

# Используем Selenium для загрузки страницы и переключения пагинации
driver = web_driver()
driver.get(base_url)
logging.info(f"Открыт URL: {base_url}")

while True:
    try:
        # Получаем HTML текущей страницы через Selenium
        page_source = driver.page_source

        # Используем BeautifulSoup для извлечения данных
        soup = BeautifulSoup(page_source, 'html.parser')

        # Ищем все карточки ресторанов
        for card in soup.find_all('div', class_='SXYe3u6-sdM- WiK1XyoVciU-'):
            # Извлекаем название ресторана
            name = card.find('h3', class_='frfPm7XUUrg- _-7pM-Oc3cZg-')
            name = name.text.strip() if name else "Название не найдено"

            # Извлекаем ценовой диапазон
            price = card.find('span', class_='fAwKcPtLqSo-')
            price = price.find(text=True, recursive=False).strip() if price else "Ценовой диапазон не найден"

            # Извлекаем ссылку на ресторан
            link = card.find('a', href=True)
            link = link['href'] if link and '/r/' in link['href'] else None

            # Добавляем данные в список
            if link:
                restaurants_data.append({
                    "name": name,
                    "price": price,
                    "link": link
                })

        logging.info(f"Найдено ресторанов на текущей странице: {len(restaurants_data)}")

        next_button = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Go to the next page']")
        # Кликаем на кнопку "->"
        next_button.click()
        logging.info("Переход на следующую страницу")
        time.sleep(3)  # Ждем загрузки следующей страницы
    except Exception as e:
        logging.error(f"Ошибка: {e}")
        break

logging.info(f"Всего найдено ресторанов: {len(restaurants_data)}")

driver.quit()

<ipython-input-41-9e97da91c53b>:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  price = price.find(text=True, recursive=False).strip() if price else "Ценовой диапазон не найден"


In [42]:
restaurants_data

[{'name': 'Macellaio RC Soho',
  'price': '£££',
  'link': 'https://www.opentable.co.uk/r/macellaio-rc-soho-london?originId=2954411d-2de8-4162-92fe-56132facbbe5&corrid=2954411d-2de8-4162-92fe-56132facbbe5&avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&p=2&sd=2025-03-02T16%3A30%3A00'},
 {'name': 'Shack-Fuyu',
  'price': '£££',
  'link': 'https://www.opentable.co.uk/r/shackfuyu-london?originId=2954411d-2de8-4162-92fe-56132facbbe5&corrid=2954411d-2de8-4162-92fe-56132facbbe5&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2025-03-02T16%3A30%3A00'},
 {'name': 'Hachi Japanese BBQ Restaurant',
  'price': '££££',
  'link': 'https://www.opentable.co.uk/r/hachi-japanese-bbq-restaurant-london?originId=2954411d-2de8-4162-92fe-56132facbbe5&corrid=2954411d-2de8-4162-92fe-56132facbbe5&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2025-03-02T16%3A30%3A00'},
 {'name': 'Mildreds Soho',
  'price': '££',
  'link': 'https://www.opentable.co.uk/r/mildreds-soho-london?originId=2954411d-2de8-416

In [37]:
import pandas as pd

In [43]:
opentable_rest = pd.DataFrame(restaurants_data)
opentable_rest

,name,price,link
0,Macellaio RC Soho,£££,https://www.opentable.co.uk/r/macellaio-rc-soh...
1,Shack-Fuyu,£££,https://www.opentable.co.uk/r/shackfuyu-london...
2,Hachi Japanese BBQ Restaurant,££££,https://www.opentable.co.uk/r/hachi-japanese-b...
3,Mildreds Soho,££,https://www.opentable.co.uk/r/mildreds-soho-lo...
4,Whitcomb's,£££,https://www.opentable.co.uk/r/whitcombs-london...
...,...,...,...
427,Tastes at Riu Plaza Hotel,££,https://www.opentable.co.uk/r/tastes-at-riu-pl...
428,Tabac,£££,https://www.opentable.co.uk/r/tabac-london?ori...
429,Crazy Pizza Knightsbridge,££££,https://www.opentable.co.uk/r/crazy-pizza-knig...
430,Angus Steakhouse Bond Street,££,https://www.opentable.co.uk/r/angus-steakhouse...


In [45]:
opentable_rest.to_csv('opentable_rest.csv', index=False)

## **Выгрузка детальной информации со страниц ресторанов**

In [56]:
# Функция для получения HTML-кода страницы
def get_page(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Проверка на ошибки HTTP
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка при запросе {url}: {e}")
        return None

In [57]:
# Функция для извлечения детальных данных
def get_restaurant_details(url):
    logging.info(f"Обрабатывается ресторан по ссылке: {url}")
    html = get_page(url)
    if not html:
        return None

    soup = BeautifulSoup(html, 'html.parser')
    details = {"link": url}

    # Количество отзывов
    reviews = soup.find('span', text=lambda x: x and 'Reviews' in x)
    if reviews:
        details["reviews"] = reviews.text.strip()
    else:
        details["reviews"] = None
    logging.info(f"Отзывы извлечены: {details['reviews']}")

    # Рейтинг
    rating = soup.find('span', class_='m1KNa9XKCHY- C7Tp-bANpE4-')
    details["rating"] = rating.text.strip() if rating else None
    logging.info(f"Рейтинг извлечен: {details['rating']}")

    # Кухня
    cuisine = soup.find('span', text="Cuisines")
    if cuisine:
        cuisine_text = cuisine.find_next('div', class_='-XkftahGV5Y-')
        details["cuisine"] = cuisine_text.text.strip() if cuisine_text else None
    else:
        details["cuisine"] = None
    logging.info(f"Типы кухни извлечены: {details['cuisine']}")

    # Дополнительные опции
    additional = soup.find('span', text="Additional")
    if additional:
        additional_text = additional.find_next('div', class_='-XkftahGV5Y-')
        details["additional"] = additional_text.text.strip() if additional_text else None
    else:
        details["additional"] = None
    logging.info(f"Дополнительные опции извлечены: {details['additional']}")

    return details

In [58]:
# Сбор данных по всем ресторанам
opentable_rest_details = []
for link in opentable_rest['link']:
    details = get_restaurant_details(link)
    if details:
        opentable_rest_details.append(details)

# Создание нового датафрейма
df_opentable_rest_details = pd.DataFrame(opentable_rest_details)
df_opentable_rest_details

<ipython-input-57-1f767bddfb91>:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  reviews = soup.find('span', text=lambda x: x and 'Reviews' in x)
<ipython-input-57-1f767bddfb91>:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  cuisine = soup.find('span', text="Cuisines")
<ipython-input-57-1f767bddfb91>:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  additional = soup.find('span', text="Additional")


,link,reviews,rating,cuisine,additional
0,https://www.opentable.co.uk/r/macellaio-rc-soh...,169 Reviews,4.2,"Steak, Italian, Contemporary Italian","Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea..."
1,https://www.opentable.co.uk/r/shackfuyu-london...,2462 Reviews,4.5,Japanese,"Beer, Cocktails, Counter Seating, Dog Friendly..."
2,https://www.opentable.co.uk/r/hachi-japanese-b...,707 Reviews,4.7,"Japanese, Barbecue","Beer, Cocktails, Non-Smoking, Takeaway, Wine"
3,https://www.opentable.co.uk/r/mildreds-soho-lo...,313 Reviews,4.4,"Vegan, Vegetarian, Vegetarian / Vegan","Beer, Cocktails, Delivery, Dog Friendly, Full ..."
4,https://www.opentable.co.uk/r/whitcombs-london...,835 Reviews,4.4,"Mediterranean, French","Bar/Lounge, Beer, Cocktails, Corkage Fee, Late..."
...,...,...,...,...,...
427,https://www.opentable.co.uk/r/tastes-at-riu-pl...,4 Reviews,4.5,"Fusion / Eclectic, International, Dining Bar",None
428,https://www.opentable.co.uk/r/tabac-london?ori...,10 Reviews,4.3,"French, Wine Bar","Bar/Lounge, Beer, Cocktails, Dog Friendly, Out..."
429,https://www.opentable.co.uk/r/crazy-pizza-knig...,68 Reviews,3.8,"Contemporary Italian, Pizzeria, Italian","Banquet, Beer, Cocktails, Delivery, Entertainm..."
430,https://www.opentable.co.uk/r/angus-steakhouse...,5 Reviews,4,"Steakhouse, Farm-to-table, Halal","Beer, Cocktails, Gluten-free Options, Late Nig..."


In [61]:
df_opentable_rest_details['link'].nunique()

432

In [62]:
df_opentable_rest_details.to_csv('opentable_rest_details.csv', index=False)

**Соединение таблиц с базовой и детальной информацией**

In [65]:
opentable_rest_data = pd.merge(opentable_rest, df_opentable_rest_details, on='link')
opentable_rest_data

,name,price,link,reviews,rating,cuisine,additional
0,Macellaio RC Soho,£££,https://www.opentable.co.uk/r/macellaio-rc-soh...,169 Reviews,4.2,"Steak, Italian, Contemporary Italian","Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea..."
1,Shack-Fuyu,£££,https://www.opentable.co.uk/r/shackfuyu-london...,2462 Reviews,4.5,Japanese,"Beer, Cocktails, Counter Seating, Dog Friendly..."
2,Hachi Japanese BBQ Restaurant,££££,https://www.opentable.co.uk/r/hachi-japanese-b...,707 Reviews,4.7,"Japanese, Barbecue","Beer, Cocktails, Non-Smoking, Takeaway, Wine"
3,Mildreds Soho,££,https://www.opentable.co.uk/r/mildreds-soho-lo...,313 Reviews,4.4,"Vegan, Vegetarian, Vegetarian / Vegan","Beer, Cocktails, Delivery, Dog Friendly, Full ..."
4,Whitcomb's,£££,https://www.opentable.co.uk/r/whitcombs-london...,835 Reviews,4.4,"Mediterranean, French","Bar/Lounge, Beer, Cocktails, Corkage Fee, Late..."
...,...,...,...,...,...,...,...
427,Tastes at Riu Plaza Hotel,££,https://www.opentable.co.uk/r/tastes-at-riu-pl...,4 Reviews,4.5,"Fusion / Eclectic, International, Dining Bar",None
428,Tabac,£££,https://www.opentable.co.uk/r/tabac-london?ori...,10 Reviews,4.3,"French, Wine Bar","Bar/Lounge, Beer, Cocktails, Dog Friendly, Out..."
429,Crazy Pizza Knightsbridge,££££,https://www.opentable.co.uk/r/crazy-pizza-knig...,68 Reviews,3.8,"Contemporary Italian, Pizzeria, Italian","Banquet, Beer, Cocktails, Delivery, Entertainm..."
430,Angus Steakhouse Bond Street,££,https://www.opentable.co.uk/r/angus-steakhouse...,5 Reviews,4,"Steakhouse, Farm-to-table, Halal","Beer, Cocktails, Gluten-free Options, Late Nig..."


# **Подготовка к сoncat с датасетом Tripadvisor + Yelp**

Нужно обработать строки так, чтобы в новый столбец `'diets'` записывались значения `Gluten-free Options, Vegeterian, Vegan`, если они содержатся либо в столбце `'additional'`, либо в `'cuisine'`. При этом, если таковые нашлись, то из исходного столбца они удаляются.

Если в столбце `'additional'` есть значение `Delivery`, то в новом столбце `'has_delivery'` ставится `True`. Из исходного столбца значение удаляется

In [70]:
# Функция для обработки строк
def process_row(row):
    # 1. Обработка has_delivery
    if row['additional'] and 'Delivery' in row['additional']:
        row['has_delivery'] = True
        row['additional'] = row['additional'].replace('Delivery', '').strip(', ')
    else:
        row['has_delivery'] = False

    # 2. Обработка diets
    diets = []
    for diet in ['Vegan', 'Vegetarian', 'Gluten-free Options']:
        if row['cuisine'] and diet in row['cuisine']:
            diets.append(diet)
            row['cuisine'] = row['cuisine'].replace(diet, '').strip(', ')
        if row['additional'] and diet in row['additional']:
            diets.append(diet)
            row['additional'] = row['additional'].replace(diet, '').strip(', ')
    row['diets'] = ', '.join(diets) if diets else None

    # Очистка пустых значений
    row['cuisine'] = row['cuisine'] if row['cuisine'] and row['cuisine'].strip() else None
    row['additional'] = row['additional'] if row['additional'] and row['additional'].strip() else None

    return row

In [80]:
# Применяем функцию к каждой строке
opentable_rest_data_upd = opentable_rest_data.apply(process_row, axis=1)

opentable_rest_data_upd

,name,price,link,reviews,rating,cuisine,additional,has_delivery,diets
0,Macellaio RC Soho,£££,https://www.opentable.co.uk/r/macellaio-rc-soh...,169 Reviews,4.2,"Steak, Italian, Contemporary Italian","Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea...",False,Gluten-free Options
1,Shack-Fuyu,£££,https://www.opentable.co.uk/r/shackfuyu-london...,2462 Reviews,4.5,Japanese,"Beer, Cocktails, Counter Seating, Dog Friendly...",False,"Vegan, Gluten-free Options"
2,Hachi Japanese BBQ Restaurant,££££,https://www.opentable.co.uk/r/hachi-japanese-b...,707 Reviews,4.7,"Japanese, Barbecue","Beer, Cocktails, Non-Smoking, Takeaway, Wine",False,None
3,Mildreds Soho,££,https://www.opentable.co.uk/r/mildreds-soho-lo...,313 Reviews,4.4,/,"Beer, Cocktails, , Dog Friendly, Full Bar, , N...",True,"Vegan, Vegan, Vegetarian, Gluten-free Options"
4,Whitcomb's,£££,https://www.opentable.co.uk/r/whitcombs-london...,835 Reviews,4.4,"Mediterranean, French","Bar/Lounge, Beer, Cocktails, Corkage Fee, Late...",False,Vegan
...,...,...,...,...,...,...,...,...,...
427,Tastes at Riu Plaza Hotel,££,https://www.opentable.co.uk/r/tastes-at-riu-pl...,4 Reviews,4.5,"Fusion / Eclectic, International, Dining Bar",None,False,None
428,Tabac,£££,https://www.opentable.co.uk/r/tabac-london?ori...,10 Reviews,4.3,"French, Wine Bar","Bar/Lounge, Beer, Cocktails, Dog Friendly, Out...",False,None
429,Crazy Pizza Knightsbridge,££££,https://www.opentable.co.uk/r/crazy-pizza-knig...,68 Reviews,3.8,"Contemporary Italian, Pizzeria, Italian","Banquet, Beer, Cocktails, , Entertainment, Ful...",True,None
430,Angus Steakhouse Bond Street,££,https://www.opentable.co.uk/r/angus-steakhouse...,5 Reviews,4,"Steakhouse, Farm-to-table, Halal","Beer, Cocktails, , Late Night, Patio/Outdoor D...",False,"Vegan, Gluten-free Options"


In [72]:
import numpy as np

Уберем лишние пробелы, запятые, заменим пустые значения на `np.nan` и сделаем другие преобразования

In [81]:
opentable_rest_data_upd2 = opentable_rest_data_upd.copy()

opentable_rest_data_upd2 = opentable_rest_data_upd2.replace(r'\s*,\s*', ', ', regex=True)  # Заменяем пробелы вокруг запятых
opentable_rest_data_upd2 = opentable_rest_data_upd2.replace(r'\s*$', '', regex=True)  # Удаляем пробелы в конце строк
opentable_rest_data_upd2 = opentable_rest_data_upd2.replace(r',\s*$', '', regex=True)  # Удаляем запятые в конце строк

# 2. Замена 'None' и '/' на np.nan
opentable_rest_data_upd2 = opentable_rest_data_upd2.replace(['None', '/'], np.nan)

# 3. Удаление пустых значений
opentable_rest_data_upd2 = opentable_rest_data_upd2.replace('', np.nan)

# Вывод результата
opentable_rest_data_upd2

,name,price,link,reviews,rating,cuisine,additional,has_delivery,diets
0,Macellaio RC Soho,£££,https://www.opentable.co.uk/r/macellaio-rc-soh...,169 Reviews,4.2,"Steak, Italian, Contemporary Italian","Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea...",False,Gluten-free Options
1,Shack-Fuyu,£££,https://www.opentable.co.uk/r/shackfuyu-london...,2462 Reviews,4.5,Japanese,"Beer, Cocktails, Counter Seating, Dog Friendly...",False,"Vegan, Gluten-free Options"
2,Hachi Japanese BBQ Restaurant,££££,https://www.opentable.co.uk/r/hachi-japanese-b...,707 Reviews,4.7,"Japanese, Barbecue","Beer, Cocktails, Non-Smoking, Takeaway, Wine",False,None
3,Mildreds Soho,££,https://www.opentable.co.uk/r/mildreds-soho-lo...,313 Reviews,4.4,NaN,"Beer, Cocktails, , Dog Friendly, Full Bar, , N...",True,"Vegan, Vegan, Vegetarian, Gluten-free Options"
4,Whitcomb's,£££,https://www.opentable.co.uk/r/whitcombs-london...,835 Reviews,4.4,"Mediterranean, French","Bar/Lounge, Beer, Cocktails, Corkage Fee, Late...",False,Vegan
...,...,...,...,...,...,...,...,...,...
427,Tastes at Riu Plaza Hotel,££,https://www.opentable.co.uk/r/tastes-at-riu-pl...,4 Reviews,4.5,"Fusion / Eclectic, International, Dining Bar",None,False,None
428,Tabac,£££,https://www.opentable.co.uk/r/tabac-london?ori...,10 Reviews,4.3,"French, Wine Bar","Bar/Lounge, Beer, Cocktails, Dog Friendly, Out...",False,None
429,Crazy Pizza Knightsbridge,££££,https://www.opentable.co.uk/r/crazy-pizza-knig...,68 Reviews,3.8,"Contemporary Italian, Pizzeria, Italian","Banquet, Beer, Cocktails, , Entertainment, Ful...",True,None
430,Angus Steakhouse Bond Street,££,https://www.opentable.co.uk/r/angus-steakhouse...,5 Reviews,4,"Steakhouse, Farm-to-table, Halal","Beer, Cocktails, , Late Night, Patio/Outdoor D...",False,"Vegan, Gluten-free Options"


In [85]:
opentable_rest_data_upd3 = opentable_rest_data_upd2.copy()

opentable_rest_data_upd3['additional'] = (
    opentable_rest_data_upd2['additional']
    .str.replace(r',\s*,', ',', regex=True)  # Удаляем запятые, окруженные пробелами
    .str.strip()  # Удаляем пробелы в начале и конце строки
)

opentable_rest_data_upd3['reviews'] = opentable_rest_data_upd2['reviews'].str.replace(' Reviews', '')
opentable_rest_data_upd3['reviews'] = pd.to_numeric(opentable_rest_data_upd3['reviews'], errors='coerce')

opentable_rest_data_upd3

,name,price,link,reviews,rating,cuisine,additional,has_delivery,diets
0,Macellaio RC Soho,£££,https://www.opentable.co.uk/r/macellaio-rc-soh...,169.0,4.2,"Steak, Italian, Contemporary Italian","Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea...",False,Gluten-free Options
1,Shack-Fuyu,£££,https://www.opentable.co.uk/r/shackfuyu-london...,2462.0,4.5,Japanese,"Beer, Cocktails, Counter Seating, Dog Friendly...",False,"Vegan, Gluten-free Options"
2,Hachi Japanese BBQ Restaurant,££££,https://www.opentable.co.uk/r/hachi-japanese-b...,707.0,4.7,"Japanese, Barbecue","Beer, Cocktails, Non-Smoking, Takeaway, Wine",False,None
3,Mildreds Soho,££,https://www.opentable.co.uk/r/mildreds-soho-lo...,313.0,4.4,NaN,"Beer, Cocktails, Dog Friendly, Full Bar, Non-S...",True,"Vegan, Vegan, Vegetarian, Gluten-free Options"
4,Whitcomb's,£££,https://www.opentable.co.uk/r/whitcombs-london...,835.0,4.4,"Mediterranean, French","Bar/Lounge, Beer, Cocktails, Corkage Fee, Late...",False,Vegan
...,...,...,...,...,...,...,...,...,...
427,Tastes at Riu Plaza Hotel,££,https://www.opentable.co.uk/r/tastes-at-riu-pl...,4.0,4.5,"Fusion / Eclectic, International, Dining Bar",None,False,None
428,Tabac,£££,https://www.opentable.co.uk/r/tabac-london?ori...,10.0,4.3,"French, Wine Bar","Bar/Lounge, Beer, Cocktails, Dog Friendly, Out...",False,None
429,Crazy Pizza Knightsbridge,££££,https://www.opentable.co.uk/r/crazy-pizza-knig...,68.0,3.8,"Contemporary Italian, Pizzeria, Italian","Banquet, Beer, Cocktails, Entertainment, Full ...",True,None
430,Angus Steakhouse Bond Street,££,https://www.opentable.co.uk/r/angus-steakhouse...,5.0,4,"Steakhouse, Farm-to-table, Halal","Beer, Cocktails, Late Night, Patio/Outdoor Din...",False,"Vegan, Gluten-free Options"


In [83]:
opentable_rest_data_upd3.dtypes

,0
name,object
price,object
link,object
reviews,object
rating,object
cuisine,object
additional,object
has_delivery,bool
diets,object


In [86]:
opentable_rest_data_upd4 = opentable_rest_data_upd3.copy()

opentable_rest_data_upd4['reviews'] = opentable_rest_data_upd3['reviews'].fillna(0).astype(int)
opentable_rest_data_upd4['rating'] = opentable_rest_data_upd3['rating'].fillna(0).astype(float)
opentable_rest_data_upd4

,name,price,link,reviews,rating,cuisine,additional,has_delivery,diets
0,Macellaio RC Soho,£££,https://www.opentable.co.uk/r/macellaio-rc-soh...,169,4.2,"Steak, Italian, Contemporary Italian","Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea...",False,Gluten-free Options
1,Shack-Fuyu,£££,https://www.opentable.co.uk/r/shackfuyu-london...,2462,4.5,Japanese,"Beer, Cocktails, Counter Seating, Dog Friendly...",False,"Vegan, Gluten-free Options"
2,Hachi Japanese BBQ Restaurant,££££,https://www.opentable.co.uk/r/hachi-japanese-b...,707,4.7,"Japanese, Barbecue","Beer, Cocktails, Non-Smoking, Takeaway, Wine",False,None
3,Mildreds Soho,££,https://www.opentable.co.uk/r/mildreds-soho-lo...,313,4.4,NaN,"Beer, Cocktails, Dog Friendly, Full Bar, Non-S...",True,"Vegan, Vegan, Vegetarian, Gluten-free Options"
4,Whitcomb's,£££,https://www.opentable.co.uk/r/whitcombs-london...,835,4.4,"Mediterranean, French","Bar/Lounge, Beer, Cocktails, Corkage Fee, Late...",False,Vegan
...,...,...,...,...,...,...,...,...,...
427,Tastes at Riu Plaza Hotel,££,https://www.opentable.co.uk/r/tastes-at-riu-pl...,4,4.5,"Fusion / Eclectic, International, Dining Bar",None,False,None
428,Tabac,£££,https://www.opentable.co.uk/r/tabac-london?ori...,10,4.3,"French, Wine Bar","Bar/Lounge, Beer, Cocktails, Dog Friendly, Out...",False,None
429,Crazy Pizza Knightsbridge,££££,https://www.opentable.co.uk/r/crazy-pizza-knig...,68,3.8,"Contemporary Italian, Pizzeria, Italian","Banquet, Beer, Cocktails, Entertainment, Full ...",True,None
430,Angus Steakhouse Bond Street,££,https://www.opentable.co.uk/r/angus-steakhouse...,5,4.0,"Steakhouse, Farm-to-table, Halal","Beer, Cocktails, Late Night, Patio/Outdoor Din...",False,"Vegan, Gluten-free Options"


In [87]:
opentable_rest_data_upd4.dtypes

,0
name,object
price,object
link,object
reviews,int64
rating,float64
cuisine,object
additional,object
has_delivery,bool
diets,object


Добавим новые столбцы, переименуем необходимые столбцы и изменим их порядок

In [91]:
opentable_rest_data_upd5 = opentable_rest_data_upd4.copy()

opentable_rest_data_upd5[['id', 'is_sponsored', 'latitude', 'longitude', 'is_premium',
                          'reviews_rating_1', 'reviews_rating_2', 'reviews_rating_3', 'reviews_rating_4', 'reviews_rating_5']] = np.nan

# Переименовываем конкретные столбцы
opentable_rest_data_upd5 = opentable_rest_data_upd5.rename(columns={
    "additional": "dining_options",
    "price": "price_range",
    "cuisine": "cuisines"
})

opentable_rest_data_upd5 = opentable_rest_data_upd5[['id','name','reviews','rating','price_range','is_sponsored',
                                                     'latitude','longitude','has_delivery','is_premium','cuisines',
                                                     'diets','dining_options','reviews_rating_1','reviews_rating_2',
                                                     'reviews_rating_3','reviews_rating_4','reviews_rating_5']]

opentable_rest_data_upd5

,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
0,NaN,Macellaio RC Soho,169,4.2,£££,NaN,NaN,NaN,False,NaN,"Steak, Italian, Contemporary Italian",Gluten-free Options,"Bar/Lounge, Beer, Cafe, Cocktails, Counter Sea...",NaN,NaN,NaN,NaN,NaN
1,NaN,Shack-Fuyu,2462,4.5,£££,NaN,NaN,NaN,False,NaN,Japanese,"Vegan, Gluten-free Options","Beer, Cocktails, Counter Seating, Dog Friendly...",NaN,NaN,NaN,NaN,NaN
2,NaN,Hachi Japanese BBQ Restaurant,707,4.7,££££,NaN,NaN,NaN,False,NaN,"Japanese, Barbecue",None,"Beer, Cocktails, Non-Smoking, Takeaway, Wine",NaN,NaN,NaN,NaN,NaN
3,NaN,Mildreds Soho,313,4.4,££,NaN,NaN,NaN,True,NaN,NaN,"Vegan, Vegan, Vegetarian, Gluten-free Options","Beer, Cocktails, Dog Friendly, Full Bar, Non-S...",NaN,NaN,NaN,NaN,NaN
4,NaN,Whitcomb's,835,4.4,£££,NaN,NaN,NaN,False,NaN,"Mediterranean, French",Vegan,"Bar/Lounge, Beer, Cocktails, Corkage Fee, Late...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,NaN,Tastes at Riu Plaza Hotel,4,4.5,££,NaN,NaN,NaN,False,NaN,"Fusion / Eclectic, International, Dining Bar",None,None,NaN,NaN,NaN,NaN,NaN
428,NaN,Tabac,10,4.3,£££,NaN,NaN,NaN,False,NaN,"French, Wine Bar",None,"Bar/Lounge, Beer, Cocktails, Dog Friendly, Out...",NaN,NaN,NaN,NaN,NaN
429,NaN,Crazy Pizza Knightsbridge,68,3.8,££££,NaN,NaN,NaN,True,NaN,"Contemporary Italian, Pizzeria, Italian",None,"Banquet, Beer, Cocktails, Entertainment, Full ...",NaN,NaN,NaN,NaN,NaN
430,NaN,Angus Steakhouse Bond Street,5,4.0,££,NaN,NaN,NaN,False,NaN,"Steakhouse, Farm-to-table, Halal","Vegan, Gluten-free Options","Beer, Cocktails, Late Night, Patio/Outdoor Din...",NaN,NaN,NaN,NaN,NaN


# **Concat с датасетом Tripadvisor + Yelp**

In [92]:
from google.colab import files
uploaded = files.upload()

Saving tripadvisor_yelp_data_final.csv to tripadvisor_yelp_data_final.csv


In [93]:
tripadvisor_yelp_data_final = pd.read_csv('tripadvisor_yelp_data_final.csv')
tripadvisor_yelp_data_final

,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
0,2418348,Leicester Arms,602,4.5,$$ - $$$,True,51.510597,-0.137677,False,False,"British, Pub","Vegetarian friendly, Gluten free options","Reservations, Seating, Street Parking, Televis...",30,23,51,156,342
1,26452946,Swiss Butter,182,4.0,$$ - $$$,True,51.520620,-0.122808,False,True,"French, Steakhouse",NaN,"Seating, Wheelchair Accessible, Serves Alcohol...",21,15,14,22,110
2,25151198,Florattica Rooftop Bar London,316,4.5,$$ - $$$,False,51.512955,-0.075292,False,True,"Bar, European, British, Healthy, Dining bars",NaN,"Reservations, Outdoor Seating, Seating, Serves...",15,4,7,11,279
3,2213156,The Black Dog Freehouse,733,4.5,$$ - $$$,False,51.488724,-0.120889,False,False,"Bar, British, Pub","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Outdoor Seating, Seating, Highch...",19,23,47,216,428
4,27930238,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"Steakhouse, British, Contemporary",Vegetarian friendly,"Reservations, Seating, Wheelchair Accessible, ...",0,1,1,2,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10434,MF6Xwxep-fx_WfjpR-hmRQ,The Quince Tree,22,4.4,£££,NaN,51.522522,-0.184909,NaN,NaN,Cafes,NaN,NaN,-1,-1,-1,-1,-1
10435,V4RBUciS7AjpqkFVMtnpAQ,CLAW,5,4.6,£,NaN,51.512658,-0.136573,NaN,NaN,Seafood,NaN,NaN,-1,-1,-1,-1,-1
10436,nkI2QsxFA4eDf2iplUDpkw,Flying Horse,11,3.9,££,NaN,51.516510,-0.130739,NaN,NaN,Pubs,NaN,NaN,-1,-1,-1,-1,-1
10437,MhTjD-ikP48Uqe9LascWPg,Fitous Thai Restaurant,8,4.3,£,NaN,51.522057,-0.226599,NaN,NaN,Thai,NaN,NaN,-1,-1,-1,-1,-1


In [95]:
tripadvisor_yelp_opentable_data = pd.concat([tripadvisor_yelp_data_final, opentable_rest_data_upd5], ignore_index=True)
tripadvisor_yelp_opentable_data

,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
0,2418348,Leicester Arms,602,4.5,$$ - $$$,True,51.510597,-0.137677,False,False,"British, Pub","Vegetarian friendly, Gluten free options","Reservations, Seating, Street Parking, Televis...",30.0,23.0,51.0,156.0,342.0
1,26452946,Swiss Butter,182,4.0,$$ - $$$,True,51.520620,-0.122808,False,True,"French, Steakhouse",NaN,"Seating, Wheelchair Accessible, Serves Alcohol...",21.0,15.0,14.0,22.0,110.0
2,25151198,Florattica Rooftop Bar London,316,4.5,$$ - $$$,False,51.512955,-0.075292,False,True,"Bar, European, British, Healthy, Dining bars",NaN,"Reservations, Outdoor Seating, Seating, Serves...",15.0,4.0,7.0,11.0,279.0
3,2213156,The Black Dog Freehouse,733,4.5,$$ - $$$,False,51.488724,-0.120889,False,False,"Bar, British, Pub","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Outdoor Seating, Seating, Highch...",19.0,23.0,47.0,216.0,428.0
4,27930238,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"Steakhouse, British, Contemporary",Vegetarian friendly,"Reservations, Seating, Wheelchair Accessible, ...",0.0,1.0,1.0,2.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10866,NaN,Tastes at Riu Plaza Hotel,4,4.5,££,NaN,NaN,NaN,False,NaN,"Fusion / Eclectic, International, Dining Bar",None,None,NaN,NaN,NaN,NaN,NaN
10867,NaN,Tabac,10,4.3,£££,NaN,NaN,NaN,False,NaN,"French, Wine Bar",None,"Bar/Lounge, Beer, Cocktails, Dog Friendly, Out...",NaN,NaN,NaN,NaN,NaN
10868,NaN,Crazy Pizza Knightsbridge,68,3.8,££££,NaN,NaN,NaN,True,NaN,"Contemporary Italian, Pizzeria, Italian",None,"Banquet, Beer, Cocktails, Entertainment, Full ...",NaN,NaN,NaN,NaN,NaN
10869,NaN,Angus Steakhouse Bond Street,5,4.0,££,NaN,NaN,NaN,False,NaN,"Steakhouse, Farm-to-table, Halal","Vegan, Gluten-free Options","Beer, Cocktails, Late Night, Patio/Outdoor Din...",NaN,NaN,NaN,NaN,NaN


In [98]:
duplicated_rows = tripadvisor_yelp_opentable_data[tripadvisor_yelp_opentable_data['name'].duplicated(keep=False)]
duplicated_rows

,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
4,27930238,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"Steakhouse, British, Contemporary",Vegetarian friendly,"Reservations, Seating, Wheelchair Accessible, ...",0.0,1.0,1.0,2.0,44.0
6,17581691,Circolo Popolare,2507,4.5,$$ - $$$,False,51.517250,-0.133866,False,False,"Italian, Pizza, Mediterranean, Neapolitan, Cam...","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Seating, Serves Alcohol, Full Ba...",178.0,161.0,227.0,229.0,1713.0
13,776287,The Ivy,7427,4.0,$$$$,False,51.512833,-0.128076,False,False,"European, British","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Private Dining, Seating, Wheelch...",303.0,420.0,891.0,1803.0,4010.0
15,7345750,Sky Pod Bar,2506,4.5,$$ - $$$,False,51.511192,-0.083558,False,False,"Bar, British","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Seating, Wheelchair Accessible, ...",71.0,77.0,137.0,432.0,1788.0
20,26318360,La Mia Mamma,525,4.0,$$ - $$$,False,51.486076,-0.172162,False,False,"Italian, Mediterranean, European, Neapolitan, ...","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Seating, Highchairs Available, S...",58.0,30.0,27.0,51.0,359.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10864,NaN,The Northumberland Arms,0,4.3,££,NaN,NaN,NaN,False,NaN,"Pub, British",None,"Counter Seating, Late Night, Outdoor Smoking A...",NaN,NaN,NaN,NaN,NaN
10865,NaN,Agrodolce London,0,0.0,££,NaN,NaN,NaN,False,NaN,Italian,Vegan,"Beer, Cocktails, Wine",NaN,NaN,NaN,NaN,NaN
10866,NaN,Tastes at Riu Plaza Hotel,4,4.5,££,NaN,NaN,NaN,False,NaN,"Fusion / Eclectic, International, Dining Bar",None,None,NaN,NaN,NaN,NaN,NaN
10868,NaN,Crazy Pizza Knightsbridge,68,3.8,££££,NaN,NaN,NaN,True,NaN,"Contemporary Italian, Pizzeria, Italian",None,"Banquet, Beer, Cocktails, Entertainment, Full ...",NaN,NaN,NaN,NaN,NaN


In [100]:
tripadvisor_yelp_opentable_data.to_csv('tripadvisor_yelp_opentable_data.csv', index=False)

In [101]:
tripadvisor_yelp_opentable_data.dtypes

,0
id,object
name,object
reviews,int64
rating,float64
price_range,object
is_sponsored,object
latitude,float64
longitude,float64
has_delivery,object
is_premium,object
